In [6]:
from keras.preprocessing import image
import sys, os
import pandas as pd
import numpy as np
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.regularizers import l2
from keras.utils import np_utils
import cv2
from matplotlib import pyplot as plt

from keras.models import model_from_json
import time

### Data Collection

The Face data we used for training is taken from kaggle in Representation Learning: Facial Expression Recognition Challenge from 2013. the data consists of 48x48 pixel grayscale images of faces arranged in a row and seven categories/labels (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral) of emotions. 


In [19]:
#functions
def xyFromDataset(xT,yT):
    
    #directories of images for each 7 emotions
    category = [cat for cat in os.listdir('training_sett')]
    
    #iterate through each category to extract image data set for each emotion
    for catT in category:
        current_image_path = os.path.join('training_sett', catT)
        imageList = [i for i in os.listdir(current_image_path) if not i.startswith('.')]
        
        #read image data as piexels and prepare training variables x,y 
        for i in imageList:
            imageT = cv2.imread(os.path.join(current_image_path, i))                
            imageT = cv2.cvtColor(imageT, cv2.COLOR_BGR2GRAY)
            tI=cv2.resize(imageT,(48,48))
            trainPixels = image.img_to_array(tI)
            trainPixels = np.expand_dims(trainPixels, axis = 0)
            trainPixels /= 255  
            xT.append(np.array(trainPixels,"float32"))
            yT.append(float(catT))
    #finalizing
    xT = np.array(xT)
    yT = np.array(yT,'float32')
    yT = np_utils.to_categorical(yT, num_classes=7)

    xT -= np.mean(xT, axis=0)
    xT /= np.std(xT, axis=0)
    
    xT = xT.reshape(xT.shape[0], 48, 48, 1)
    
    offset = int(xT.shape[0]-xT.shape[0]/4)

    xTT = xT[0:offset]
    yTT = yT[0:offset]
    xV = xT[offset:]
    yV = yT[offset:]
    
    return xTT, yTT,xV,yV

#capture training data
def gatherData(source,emotions):
    
    #local variables for file name and directory name
    i=0
    label = 0
    m = "/record"
    a = str(time.time())
    
    #current emotion data save location
    categoryPath = os.path.join('training_sett', str(label))
    
    #start capturing data using opencv and load face detection dataset 'frontalface_default.xml'
    liveFrame = cv2.VideoCapture(source)
    faceDetectioin = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    #track key press
    K = None

    while True:

        #capture frame from source
        val, img = liveFrame.read()
        
        #if no frame is captured continue
        if not val:
            continue;
        
        #convert captured image into grayscale for management ease
        imageT = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        #invoke face detection on the captured frame
        detectedFaces = faceDetectioin.detectMultiScale(imageT, 1.32, 5)
        
        #loop through detected face list and save each face to the corresponding emotion directory label
        for (x,y,w,h) in detectedFaces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,128,0),thickness=4)
            cv2.putText(img, emotions[label] + m, (int(x), int(y)), cv2.FONT_HERSHEY_DUPLEX, 1, (120,0,255), 2)
            
            if K == ord('w') or m == "/recording":    
                
                m = "/recording"
                cp = os.path.join(categoryPath, a + str(i) + ".jpg")
                predImg=imageT[y:y+w,x:x+h]
                predImg=cv2.resize(predImg,(48,48))
                cv2.imwrite(cp, predImg)
                i+=1
                print (cp)
                
            if K == ord('q') and m == "/recording":
                
                m = "/record"
                if(label < 6):
                    label+=1
                    categoryPath = os.path.join('training_sett', str(label))
                else:
                    break 
                
                
                
                
                
        #open openCV's display window with specified frame size
        resized_img = cv2.resize(img, (1000, 700))
        cv2.imshow('Collecting Data',resized_img)
        K = cv2.waitKey(10) & 0xFF
        
        #close openCV's window if key 'e' is pressed
        if K == ord('e'):
            break

    liveFrame.release()
    cv2.destroyAllWindows
    
    
#user fer 2013 instead
def xyFromDatasetFer2013():
    #prepare training variables using fer2013 dataset
    #load the face data using panda from csv and extract triaining data from it.
    faceData = pd.read_csv('fer2013.csv').iterrows()

    trainY = []
    trainX = []

    textY = []
    testX = []

    for i,j in faceData:

        if 'Training' in j['Usage']:
            trainX.append(np.array(j['pixels'].split(" "),'float32'))
            trainY.append(j['emotion'])

        elif 'PublicTest' in j['Usage']:
            testX.append(np.array(j['pixels'].split(" "),'float32'))
            textY.append(j['emotion'])
    #flatten the data array and normalize between 1 and 0 using mean and standard deviation for x

    trainX = np.array(trainX)
    trainY = np.array(trainY)
    trainY = np_utils.to_categorical(trainY, num_classes=7)

    trainX -= np.mean(trainX, axis=0)
    trainX /= np.std(trainX, axis=0)

    testX = np.array(testX)
    textY = np.array(textY)
    textY = np_utils.to_categorical(textY, num_classes=7)

    testX -= np.mean(testX, axis=0)
    testX /= np.std(testX, axis=0)


    #reshape the matrices to represent each face data
    trainX = trainX.reshape(trainX.shape[0], 48, 48, 1)
    testX = testX.reshape(testX.shape[0], 48, 48, 1)
    
    return trainX,trainY,testX,textY

In [20]:
#compie and fit the CNN model
def comFit(model,trainX, trainY,epoch,bs,testX=None, testY=None):
    #Compliling the model
    model.compile(loss=categorical_crossentropy,optimizer=Adam(),metrics=['accuracy'])

    #Training the model
    model.fit(trainX, trainY,epochs=epoch, batch_size=bs,verbose=1,shuffle=True, validation_data=(testX, testY),)
    

    
    return model

#real time emotion prediction from specified source
def realPred(source,model,emotions):
    liveFrame = cv2.VideoCapture(source)
    faceDetectioin = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while True:

        val, img = liveFrame.read()

        if not val:
            continue;

        testImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        detectedFaces = faceDetectioin.detectMultiScale(testImg, 1.32, 5)

        for (x,y,w,h) in detectedFaces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,128,0),thickness=4)

            predImg=testImg[y:y+w,x:x+h]
            predImg=cv2.resize(predImg,(48,48))
            predPixels = image.img_to_array(predImg)
            predPixels = np.expand_dims(predPixels, axis = 0)
            predPixels /= 255

            pridectedList = model.predict(predPixels)
           
            maxVal = np.argmax(pridectedList[0])
            predEmotion = emotions[maxVal]
            
            path = os.path.join("emot",predEmotion+".png")
          
            
            emoji = cv2.imread(path,-1)
            emoji = cv2.resize(emoji,(50,50))
            
            y1, y2 = int(y), int(y) + emoji.shape[0]
            x1, x2 = int(x), int(x) + emoji.shape[1]
            
            ealpha = emoji[:, :, 3] / 255.0
            ialpha = 1.0 - ealpha

            for c in range(0, 3):
                img[y1:y2, x1:x2, c] = (ealpha * emoji[:, :, c] + ialpha * img[y1:y2, x1:x2, c])
            
            cv2.putText(img, predEmotion, (int(x), int(y)), cv2.FONT_HERSHEY_DUPLEX, 1, (120,0,255), 2)

        resized_img = cv2.resize(img, (1000, 600))
        cv2.imshow('Analayzing Facial Emotion',resized_img)

        if cv2.waitKey(10) & 0xFF == ord('e'):
            break

    liveFrame.release()
    cv2.destroyAllWindows
    #exit()
#CNN Model from training set
def craeateModel(trainX,labels):
    
    model = Sequential()

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(trainX.shape[1:])))
    model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
    model.add(Dropout(0.5))

    '''    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))'''
    
    model.add(Flatten())

    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(labels, activation='softmax'))
    
    return model


In [21]:
#possible target emotions
emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

#training and testing data xy
xT,yT,xV,yV = [],[],[],[]

val = xyFromDataset(xT,yT)

xT = np.asarray(val[0])
yT = np.asarray(val[1])
xV = np.asarray(val[2])
yV = np.asarray(val[3])

### Feature filtering
We used a three layer Convolutional neural network for extracting features and we aplied maxpooling to reduce the dimensions of the data. Finally we flatten the image matrix and pass it through a fully connected layer to classify the images. 

since we only have one input(image pixels) and one output(the pridicted emotion, 0 to 6 ),We used sequential class from keras for creating the CNN model. We add each layer instance with conv2d method, which  creates convolution kernel(4,4) that is convolved with the layer input to produce a tensor of outputs.The activatioin function used is relu, which activiate each neuron(not all at once) to produce output for the next layer. The drop out rate used is 0.2.

MaxPolling2D is used with window size and stride of 3 in each dimension. 

In [23]:
model = craeateModel(xT,7)

###Compile and train the model

in the model configuration, with use adam optimizer and 'accuracy' as a metric to evaluate the model during the training and testing. And a loss/objective function used is crossentropy.

in the triaining of the model, number of samples per gradient update is set to be 64, max iteration is set to be 32, verbosity mode used is progress bar during training and shuffle is on , to shuffle the training data before each epoch. 



In [25]:
model =  comFit(model,xT, yT,10,32,xV,yV)

Epoch 1/10
79/79 [==============================] - 35s 442ms/step - loss: 1.3933 - accuracy: 0.5860 - val_loss: 7.5493 - val_accuracy: 0.2919
Epoch 2/10
79/79 [==============================] - 37s 465ms/step - loss: 0.0962 - accuracy: 0.9683 - val_loss: 8.3116 - val_accuracy: 0.3493
Epoch 3/10
79/79 [==============================] - 37s 470ms/step - loss: 0.0318 - accuracy: 0.9909 - val_loss: 7.9537 - val_accuracy: 0.3385
Epoch 4/10
79/79 [==============================] - 37s 471ms/step - loss: 0.0253 - accuracy: 0.9914 - val_loss: 5.1796 - val_accuracy: 0.3624
Epoch 5/10
79/79 [==============================] - 39s 490ms/step - loss: 0.0248 - accuracy: 0.9930 - val_loss: 8.1491 - val_accuracy: 0.3541
Epoch 6/10
79/79 [==============================] - 38s 485ms/step - loss: 0.0255 - accuracy: 0.9944 - val_loss: 7.3745 - val_accuracy: 0.3385
Epoch 7/10
79/79 [==============================] - 37s 473ms/step - loss: 0.0061 - accuracy: 0.9988 - val_loss: 10.1776 - val_accuracy: 0.357

###Facial Emotion Prediction

capturing live frames:
we used pc camera as source of video and we used openCV library.we used VideoCapture class with '0' as an argument for capturing videos from the default source of video(camera). We created an infinite loop to capture frames  from the vieo iteratively using the read method of VideoCapture class and convert the frame into grayscale.

detecting facial region:
We again make use of openCV's face detection api CascadeClassifier, to detect facial arias of the captured frame. The training data set we use for CascadeClassifier is haaarcascade_frontalface_default.xml. detectMultiScale returns the detected faces as list of rectangles and their locations.

prediction:
We loop through the detected face recangles and draw the rectangles on the screen. we select area of interst from the image based on the returned rectangle and the location of the image. convert the selected area into array of metrices containing pixles after resizing the image based on our training data. reduce the rgb representation by dividing it by 255 and give the image to our model for prediction.


In [26]:
#saving our model for later prediction use
modelJson = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(modelJson)
model.save_weights("model.h5")

In [27]:
#load our model for prediction
'''model = model_from_json(open("model3.json", "r").read())
model.load_weights('model3.h5')
'''
realPred(0,model,emotions)

In [30]:
# evaluation of our model
score = model.evaluate(xV, yV, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Test Loss: 9.50487995147705
Test accuracy: 0.3552631437778473
